In [ ]:
pip install ultralytics opencv-python roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="423HnFzQipKd10KD0Hzu")
project = rf.workspace("diplome").project("eggsdetect")
version = project.version(1)
dataset = version.download("yolov11")

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11s.pt")  # Lightweight and fast
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=5,
    imgsz=640,
    batch=32,
    device="cuda",  # or "cpu" if GPU not available
    name="egg_detector",
    amp=True
)

In [ ]:
import cv2
from ultralytics import YOLO
import os

# Allow duplicate OpenMP if needed
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Load model
model = YOLO("/content/runs/detect/egg_detector2/weights/best.pt")

# Video input
video_path = "/content/20180910_144521.mp4"
cap = cv2.VideoCapture(video_path)

# Read first frame for dimensions
ret, frame = cap.read()
if not ret:
    raise Exception("Failed to open video.")

frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
height, width = frame.shape[:2]
fps = cap.get(cv2.CAP_PROP_FPS) or 30

# Output writer
out = cv2.VideoWriter("output_egg_counter.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

line_y = int(height * 0.6)
egg_count = 0
counted_centers = []
frame_number = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    results = model.predict(source=frame, conf=0.5, verbose=False)
    boxes = results[0].boxes

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        if line_y - 5 <= cy <= line_y + 5:
            if cx not in counted_centers:
                egg_count += 1
                counted_centers.append(cx)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)

    cv2.line(frame, (0, line_y), (width, line_y), (0, 0, 255), 2)
    cv2.putText(frame, f"Egg Count: {egg_count}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 2)

    out.write(frame)
    frame_number += 1

cap.release()
out.release()
print(f"Total eggs counted: {egg_count}")